In [ ]:
import os
os.environ["VRE_LOGLEVEL"] = "0"
from vre.readers import MultiTaskDataset
from vre.representations import build_representations_from_cfg
from vre_data_analysis import (extract_b64_image_from_fig, histogram_from_classification_task, save_html,
                               extract_pil_from_b64_image, extract_b64_imgsrc_from_fig, gaussian_from_statistics)
from vre_repository import get_vre_repository
import numpy as np

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
data_path = "../../test/vre_repository/end_to_end/task_mapped_representations/data"
cfg_path = "../../test/vre_repository/end_to_end/task_mapped_representations/cfg.yaml"

representations = build_representations_from_cfg(cfg_path, get_vre_repository())


In [ ]:
print(representations)
reader = MultiTaskDataset(data_path, task_names=[r.name for r in representations],
                          task_types={r.name: r for r in representations}, normalization="min_max")
print(reader)
print(reader[0])

In [ ]:
imgs, imgsrcs = [], []
for classif_task in reader.classification_tasks:
    fig = histogram_from_classification_task(reader, classif_task, n=100, mode="random", figsize=(4, 3))
    display(pil_img := extract_pil_from_b64_image(extract_b64_image_from_fig(fig)))
    imgs.append(np.array(pil_img))
    imgsrcs.append(extract_b64_imgsrc_from_fig(fig))

In [ ]:

regression_tasks = [t for t in reader.tasks if t not in reader.classification_tasks]
for regression_task in regression_tasks:
    fig = gaussian_from_statistics(reader, regression_task)
    display(pil_imgs := extract_pil_from_b64_image(extract_b64_image_from_fig(fig)))
    imgsrcs.append(extract_b64_imgsrc_from_fig(fig))
    imgs.append(np.array(pil_img))

save_html(imgsrcs, str(reader), "plot.html")


In [ ]:
from PIL import Image
display(Image.fromarray(np.concatenate(imgs, axis=0)))